# Tuning Xbgoost

In [1]:
import numpy
import csv
import time
import pandas
import signal
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline

from sklearn.multioutput import MultiOutputRegressor

import xgboost

In [5]:
def lowest_correct(trues, preds):
    num_of_options = len(trues)
    drop_down_options = list(reversed(numpy.argsort(preds)))
    correct_options = [i for i in range(num_of_options) if trues[i]==1]
    return max([drop_down_options.index(correct_option) for correct_option in correct_options]) + 1
def average_lowest_correct(list_of_trues, list_of_preds):
    length = len(list_of_trues)
    return numpy.mean([lowest_correct(list(list_of_trues.iloc[i]), list(list_of_preds[i])) for i in range(length)])
def cv_ensemble(model, xs, ys):
    temp = cross_validate(MultiOutputRegressor(model), xs, ys, return_train_score=True, scoring=make_scorer(average_lowest_correct), n_jobs=1, cv=5, verbose=2)
    return numpy.mean(temp['test_score']).round(2)

## Problem Type

In [3]:
tfidf = pandas.read_csv('07 NM PT TFIDF.csv', delimiter=',', encoding='latin-1').fillna('')
tfidf[:3]

,____________________,_____________________,_____________________ follow,able,acceptable,accessory,accessory require,accidentally,accommodate,accompany,...,work,would,write,writer,wrong,wrong patient,wrong tattoo,xrt,xrt date,xrt pt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
ohe = pandas.read_csv('07 NM PT OHE.csv', delimiter=',', encoding='latin-1').fillna('')
ohe[:3]

,Wrong target or OAR contours,Treatment plan acceptable but not physically deliverable,Excess imaging dose,Failure to perform on-treatment imaging as per instructions,Wrong patient,Fall or other patient injury or medical condition,Wrong anatomical site (excluding laterality),Treatment plan (isodose distribution) unacceptable,Systematic hardware/software (including dose-volume) error,Interventional procedure error (Retired value),...,Inappropriate or poorly informed decision to treat or plan,Inadequate coordination of combined modality care,"Wrong, missing, mislabeled, or damaged treatment accessories",Wrong side (laterality),Wrong prescription dose-fractionation or calculation error,Radiation therapy scheduling error,Treatment not delivered - personnel/hardware/software failure,"Wrong patient position, setup point, or shift",Wrong planning margins,Allergic reaction
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
cv_ensemble(xgboost.XGBRegressor(), tfidf, ohe)

[CV]  ................................................................
[CV] ................................................. , total= 3.8min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.8min remaining:    0.0s


[CV]  ................................................................
[CV] ................................................. , total= 4.1min
[CV]  ................................................................
[CV] ................................................. , total= 4.4min
[CV]  ................................................................
[CV] ................................................. , total= 4.1min
[CV]  ................................................................
[CV] ................................................. , total= 4.2min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 20.8min finished


3.2

In [8]:
grd = {
    'mb__estimator__booster': ['gbtree', 'gblinear', 'dart'],
}
gsc = GridSearchCV(estimator=xgboost.XGBRegressor(), param_grid=grd, scoring=make_scorer(average_lowest_correct), n_jobs=1, cv=2, verbose=2, error_score=0)
gsc.fit(tfidf, ohe)
prn_rsl(gsc)

Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] mb__estimator__booster=gbtree ...................................


ValueError: Invalid parameter mb for estimator XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1). Check the list of available parameters with `estimator.get_params().keys()`.

### Default
1.69, MultiOutput(Ridge)
### Tuned
1.67 MultiOutput(BaggingRegressor(LinearSVR)) (k=2)

1.67 MultiOutput(BaggingRegressor(PassiveAggressiveRegressor)) (k=2)

1.57 MultiOutput(BaggingRegressor(LinearSVR)) (k=10)

## Problem Type

In [4]:
tfidf = pandas.read_csv('07 NM PT TFIDF.csv', delimiter=',', encoding='latin-1').fillna('')
tfidf[:3]

,____________________,_____________________,_____________________ follow,able,acceptable,accessory,accessory require,accidentally,accommodate,accompany,...,work,would,write,writer,wrong,wrong patient,wrong tattoo,xrt,xrt date,xrt pt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
ohe = pandas.read_csv('07 NM PT OHE.csv', delimiter=',', encoding='latin-1').fillna('')
ohe[:3]

,Wrong target or OAR contours,Treatment plan acceptable but not physically deliverable,Excess imaging dose,Failure to perform on-treatment imaging as per instructions,Wrong patient,Fall or other patient injury or medical condition,Wrong anatomical site (excluding laterality),Treatment plan (isodose distribution) unacceptable,Systematic hardware/software (including dose-volume) error,Interventional procedure error (Retired value),...,Inappropriate or poorly informed decision to treat or plan,Inadequate coordination of combined modality care,"Wrong, missing, mislabeled, or damaged treatment accessories",Wrong side (laterality),Wrong prescription dose-fractionation or calculation error,Radiation therapy scheduling error,Treatment not delivered - personnel/hardware/software failure,"Wrong patient position, setup point, or shift",Wrong planning margins,Allergic reaction
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
for mdl in mdls:
    print(cv_ensemble(mdl, tfidf, ohe), mdl)

[CV]  ................................................................
[CV] ................................................. , total=23.4min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 23.5min remaining:    0.0s


[CV] ................................................. , total=25.1min
[CV]  ................................................................
[CV] ................................................. , total=17.1min
[CV]  ................................................................
[CV] ................................................. , total=11.2min
[CV]  ................................................................
[CV] ................................................. , total=16.4min
[CV]  ................................................................
[CV] ................................................. , total= 9.7min
[CV]  ................................................................
[CV] ................................................. , total=16.5min
[CV]  ................................................................
[CV] ................................................. , total=12.7min
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 159.9min finished


[CV] ................................................. , total=  12.1s
[CV]  ................................................................


...

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  7.7min finished


[CV] ................................................. , total=  32.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   36.3s remaining:    0.0s


[CV] ................................................. , total=  33.2s
[CV]  ................................................................
[CV] ................................................. , total=  36.2s
[CV]  ................................................................
[CV] ................................................. , total=  34.0s
[CV]  ................................................................
[CV] ................................................. , total=  31.9s
[CV]  ................................................................
[CV] ................................................. , total=  34.0s
[CV]  ................................................................
[CV] ................................................. , total=  34.6s
[CV]  ................................................................
[CV] ................................................. , total=  37.8s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  6.4min finished


### Default
2.90 MultiOutputRegressor(BaggingRegressor(Ridge))
### Tuned
2.74 MultiOutputRegressor(BaggingRegressor(PassiveAggressiveRegressor)) (k=2)

2.36 MultiOutputRegressor(BaggingRegressor(PassiveAggressiveRegressor)) （k=10)

## Contributing Factors

In [4]:
tfidf = pandas.read_csv('07 NM CF TFIDF.csv', delimiter=',', encoding='latin-1').fillna('')
tfidf[:3]

,____________________,_____________________,_____________________ follow,able,acceptable,access,accessory,accessory require,accidentally,accommodate,...,wrong ct,wrong info,wrong info mri,wrong patient,wrong pt,wrong set,wrong tattoo,xrt,xrt date,xrt pt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
ohe = pandas.read_csv('07 NM CF OHE.csv', delimiter=',', encoding='latin-1').fillna('')
ohe[:3]

,Organizational and/or workspace resources inadequate (excluding human resources),Unnamed: 1,Patient or family member medical condition preference or behaviour,Equipment quality assurance and/or maintenance inadequate,Handoffs inadequate,Communication or documentation inadequate (patient specific),Policies and/or procedures non-existent or inadequate,"Patient or family member medical condition, preference or behaviour",Policies and/or procedures not followed,Unfamiliar treatment approach or radiation treatment technique,...,"Equipment software or hardware design, including 'human factors' design, inadequate",Failure to identify potential risks,"Equipment software or hardware commissioning, calibration or acceptance testing inadequate",Distraction or diversions involving staff,Change management,Human resources inadequate,Expectation bias involving staff,Staff behaviour,Staff education or training inadequate,Patient education inadequate
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
for mdl in mdls[1:]:
    print(cv_ensemble(mdl, tfidf, ohe), mdl)

[CV]  ................................................................
[CV] ................................................. , total=  24.7s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   32.0s remaining:    0.0s


[CV] ................................................. , total=  25.1s
[CV]  ................................................................
[CV] ................................................. , total=  27.6s
[CV]  ................................................................
[CV] ................................................. , total=  25.7s
[CV]  ................................................................
[CV] ................................................. , total=  24.7s
[CV]  ................................................................
[CV] ................................................. , total=  21.4s
[CV]  ................................................................
[CV] ................................................. , total=  21.7s
[CV]  ................................................................
[CV] ................................................. , total=  23.9s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  5.3min finished


[CV] ................................................. , total= 1.8min
[CV]  ................................................................


...

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 18.2min finished


[CV] ................................................. , total= 1.2min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


[CV] ................................................. , total= 1.3min
[CV]  ................................................................
[CV] ................................................. , total= 1.6min
[CV]  ................................................................
[CV] ................................................. , total= 1.5min
[CV]  ................................................................
[CV] ................................................. , total= 1.8min
[CV]  ................................................................
[CV] ................................................. , total= 1.6min
[CV]  ................................................................
[CV] ................................................. , total= 1.2min
[CV]  ................................................................
[CV] ................................................. , total= 1.2min
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 16.0min finished


## Default

    5.72
    
## Tuned:
    
    5.25 MultiOutputRegressor(BaggingRegressor(RandomForestRegressor))

## Overall Severity

In [7]:
tfidf = pandas.read_csv('07 NM OS TFIDF.csv', delimiter=',', encoding='latin-1').fillna('')
tfidf[:3]

,____________________,_____________________,_____________________ follow,able,acceptable,access,accessory,accessory require,accidentally,accommodate,...,wrong ct,wrong info,wrong info mri,wrong patient,wrong pt,wrong set,wrong tattoo,xrt,xrt date,xrt pt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
ohe = pandas.read_csv('07 NM OS OHE.csv', delimiter=',', encoding='latin-1').fillna('')
ohe[:3]

,Severe,Moderate,None,Mild
0,0,0,1,0
1,0,0,0,1
2,0,0,1,0


In [9]:
for mdl in mdls:
    print(cv_ensemble(mdl, tfidf, ohe), mdl)

[CV]  ................................................................
[CV] ................................................. , total=16.1min
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 16.1min remaining:    0.0s


[CV] ................................................. , total= 5.5min
[CV]  ................................................................
[CV] ................................................. , total=12.8min
[CV]  ................................................................
[CV] ................................................. , total= 9.0min
[CV]  ................................................................
[CV] ................................................. , total=10.3min
[CV]  ................................................................
[CV] ................................................. , total= 7.9min
[CV]  ................................................................
[CV] ................................................. , total= 2.7min
[CV]  ................................................................
[CV] ................................................. , total= 4.4min
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 92.0min finished


[CV] ................................................. , total=   5.8s
[CV]  ................................................................


...

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  3.5min finished


[CV] ................................................. , total=  13.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.7s remaining:    0.0s


[CV] ................................................. , total=  13.5s
[CV]  ................................................................
[CV] ................................................. , total=  14.1s
[CV]  ................................................................
[CV] ................................................. , total=  13.7s
[CV]  ................................................................
[CV] ................................................. , total=  17.2s
[CV]  ................................................................
[CV] ................................................. , total=  13.2s
[CV]  ................................................................
[CV] ................................................. , total=  11.7s
[CV]  ................................................................
[CV] ................................................. , total=  12.2s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.7min finished


## Tuned

1.11 MultiOutputRegressor(BaggingRegressor(LinearSVR))

1.11 MultiOutputRegressor(BaggingRegressor(PassiveAggressiveRegressor))

1.11 MultiOutputRegressor(BaggingRegressor(RandomForestRegressor))